In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 85% 280M/331M [00:00<00:00, 859MB/s] 
100% 331M/331M [00:00<00:00, 714MB/s]


In [ ]:

import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [ ]:
import os
import numpy as np
import pandas as pd

folder_path = '/content/utkface_aligned_cropped/UTKFace'

age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
    age.append(int(file.split('_')[0]))
    gender.append(int(file.split('_')[1]))
    img_path.append(file)

df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df  = df.sample(frac=1,random_state=0).iloc[20000:]


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
IMG_SIZE = (224, 224)


def load_img(row):
    img_path = tf.strings.join([folder_path, "/", row["img"]])

    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = preprocess_input(image)

    age = tf.cast(row["age"], tf.float32)
    gender = tf.cast(row["gender"], tf.float32)

    return image, {"age": age, "gender": gender}


train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df))
test_ds = tf.data.Dataset.from_tensor_slices(dict(test_df))

train_ds = train_ds.map(load_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(load_img, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.05),
])

def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label


train_ds_aug = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [ ]:
IMG_SIZE = (200, 200, 3)

inputs = Input(shape=IMG_SIZE)

# CNN feature extractor
x = Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
x = MaxPool2D()(x)

x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = MaxPool2D()(x)

x = Conv2D(32, (3,3), activation="relu", padding="same")(x)
x = MaxPool2D()(x)

x = Conv2D(32, (3,3), activation="relu", padding="same")(x)
x = MaxPool2D()(x)

# IMPORTANT FIX: Flatten CNN output
x = Flatten()(x)

# Shared Dense layer
x = Dense(256, activation="relu")(x)


# Age branch
age_dense = Dense(128, activation="relu")(x)

age_dense = Dense(64, activation="relu")(age_dense)

age_output = Dense(1, activation="relu", name="age")(age_dense)

# Gender branch
gender_dense = Dense(128, activation="relu")(x)

gender_dense = Dense(64, activation="relu")(gender_dense)

gender_output = Dense(1, activation="sigmoid", name="gender")(gender_dense)

model1 = Model(inputs, [age_output, gender_output])
model1.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 200, 200,  │        896 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 100, 100,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 100, 100,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 50, 50,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 50, 50,    │     18,464 │ max_pooling2d_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 25, 25,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 25, 25,    │      9,248 │ max_pooling2d_2[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 12, 12,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4608)      │          0 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 256)       │  1,179,904 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 128)       │     32,896 │ dense_32[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 128)       │     32,896 │ dense_32[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 64)        │      8,256 │ dense_33[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 64)        │      8,256 │ dense_35[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ age (Dense)         │ (None, 1)         │         65 │ dense_34[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gender (Dense)      │ (None, 1)         │         65 │ dense_36[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,309,442 (5.00 MB)

 Trainable params: 1,309,442 (5.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model1.compile(
    optimizer="adam",
    loss={
        "age": "mae",
        "gender": "binary_crossentropy"
    },
    metrics={
        "age": "mae",
        "gender": "accuracy"
    },
    loss_weights={"age": 1.0, "gender": 50.0}   # Gender converges slower
)

# --------------------------
#       TRAINING
# --------------------------
history1 = model1.fit(
    train_ds_aug,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 188s 287ms/step - age_loss: 17.2032 - age_mae: 17.2032 - gender_accuracy: 0.6850 - gender_loss: 0.5902 - loss: 46.7110 - val_age_loss: 12.1983 - val_age_mae: 12.2004 - val_gender_accuracy: 0.8002 - val_gender_loss: 0.4326 - val_loss: 33.8278
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 176s 281ms/step - age_loss: 12.5634 - age_mae: 12.5634 - gender_accuracy: 0.7985 - gender_loss: 0.4304 - loss: 34.0856 - val_age_loss: 10.2318 - val_age_mae: 10.2318 - val_gender_accuracy: 0.8301 - val_gender_loss: 0.3751 - val_loss: 28.9743
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 282ms/step - age_loss: 11.1816 - age_mae: 11.1816 - gender_accuracy: 0.8233 - gender_loss: 0.3799 - loss: 30.1784 - val_age_loss: 9.9263 - val_age_mae: 9.9258 - val_gender_accuracy: 0.8568 - val_gender_loss: 0.3256 - val_loss: 26.1929
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 176s 281ms/step - age_loss: 10.4547 - age_mae: 10.4547 - gender_accuracy: 0.8409 - gender_loss: 0.3484 - loss: 27.87

In [ ]:
results1 = model1.evaluate(test_ds)
gender_accuracy1 = results1[-1]
age_mae1 = results1[-2]
print("Gender Accuracy:", gender_accuracy1)
print("Age MAE:", age_mae1)

116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - age_loss: 7.7609 - age_mae: 7.7609 - gender_accuracy: 0.8922 - gender_loss: 0.2781 - loss: 21.6658
Gender Accuracy: 0.8894282579421997
Age MAE: 7.898099422454834


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model
conv_base = ResNet50(include_top=False, input_shape=(224,224,3))

# Freeze all layers except last conv block
set_trainable = False
for layer in conv_base.layers:
    if "conv5" in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False


x = conv_base.output
x = GlobalAveragePooling2D()(x)

# Age branch
d1 = Dense(512, activation='relu')(x)
d1 = BatchNormalization()(d1)

d1 = Dense(256, activation='relu')(d1)

d1 = Dense(128, activation='relu')(d1)

d1 = Dense(64, activation='relu')(d1)
d1 = Dropout(0.1)(d1)
age_output = Dense(1, activation='linear', name='age')(d1)

# Gender branch
d2 = Dense(512, activation='relu')(x)
d2 = BatchNormalization()(d2)

d2 = Dense(256, activation='relu')(d2)
d2= Dropout(0.2)(d2)
d2 = Dense(128, activation='relu')(d2)
d2= Dropout(0.2)(d2)
d2 = Dense(64, activation='relu')(d2)

gender_output = Dense(1, activation='sigmoid', name='gender')(d2)

model = Model(inputs=conv_base.input, outputs=[age_output, gender_output])
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 26,035,074 (99.32 MB)

 Trainable params: 17,421,314 (66.46 MB)

 Non-trainable params: 8,613,760 (32.86 MB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss={
        "age": tf.keras.losses.Huber(delta=5.0),
        "gender": "binary_crossentropy"
    },
    loss_weights={
        "age": 2.0,       # prioritize age
        "gender": 1.0
    },
    metrics={
        "age": "mae",
        "gender": "accuracy"
    }
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',        # monitor validation loss
    patience=3,                # stop if no improvement for 5 epochs
    restore_best_weights=True, # VERY important
    verbose=1
)

history = model.fit(
    train_ds_aug,
    epochs=25,
    validation_data=test_ds,
    callbacks=[early_stop]
)



Epoch 1/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 276s 397ms/step - age_loss: 11.4486 - age_mae: 4.0006 - gender_accuracy: 0.9353 - gender_loss: 0.1564 - loss: 23.0537 - val_age_loss: 15.3569 - val_age_mae: 4.8691 - val_gender_accuracy: 0.9334 - val_gender_loss: 0.1792 - val_loss: 30.8782
Epoch 2/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 241s 385ms/step - age_loss: 11.1224 - age_mae: 3.9251 - gender_accuracy: 0.9369 - gender_loss: 0.1531 - loss: 22.3979 - val_age_loss: 15.3364 - val_age_mae: 4.8691 - val_gender_accuracy: 0.9342 - val_gender_loss: 0.1791 - val_loss: 30.8369
Epoch 3/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 240s 384ms/step - age_loss: 11.0607 - age_mae: 3.9092 - gender_accuracy: 0.9381 - gender_loss: 0.1534 - loss: 22.2749 - val_age_loss: 15.3828 - val_age_mae: 4.8793 - val_gender_accuracy: 0.9312 - val_gender_loss: 0.1800 - val_loss: 30.9297
Epoch 4/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 256s 374ms/step - age_loss: 10.9568 - age_mae: 3.8949 - gender_accuracy: 0.9364 - gender_loss: 0.1551 - loss: 22.0687 - 

In [ ]:
results = model.evaluate(test_ds)
gender_accuracy = results[-1]
age_mae = results[-2]
print("Gender Accuracy:", gender_accuracy)
print("Age MAE:", age_mae)


116/116 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - age_loss: 15.1239 - age_mae: 4.8178 - gender_accuracy: 0.9298 - gender_loss: 0.1863 - loss: 30.4338
Gender Accuracy: 0.9341963529586792
Age MAE: 4.869068622589111
